In [24]:
from dotenv import load_dotenv
from openai import OpenAI

import os
import json
import requests
import gradio as gr
from openai.types.responses import Response

### Load environment variables and intialize OpenAI client

In [25]:
load_dotenv()
client = OpenAI()

### Prepare input files for the prompt

In [26]:
resume_pdf_file = client.files.create(
    file=open("data/resume.pdf", "rb"),
    purpose="user_data"
)
linkedin_pdf_file = client.files.create(
    file=open("data/linkedin.pdf", "rb"),
    purpose="user_data"
)

### Prepare tools for the prompt

In [27]:
def notify(title: str, message: str):
    return requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_KEY"),
            "user": os.getenv("PUSHOVER_EMAIL"),
            "title": title,
            "message": message,
        }
    )


def record_unknown_question(question: str) -> str:
    notify(title="Unknown question recorded", message=question)
    return "recorded_successfully"


def record_user_details(email: str, name="Unknown", notes="NA") -> str:
    notify(title="User details recorded", message=f"Email: {email}\nName: {name}\nNotes: {notes}")
    return "recorded_successfully"


tools = [
    {
        "type": "function",
        "name": "record_unknown_question",
        "description": "Record an unknown question asked by the user.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The question that the user asked",
                },
            },
            "required": ["question"],
        },
    },
    {
        "type": "function",
        "name": "record_user_details",
        "description": "Record the user's details to reach out to them later.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the user",
                },
                "email": {
                    "type": "string",
                    "description": "The email of the user",
                },
                "notes": {
                    "type": "string",
                    "description": "Any additional notes for context",
                },
            },
            "required": ["email"],
        },
    },
]

### Prepare helpers to call OpenAI

In [ ]:
from openai.types.responses import ResponseCustomToolCall
from openai.types.responses.response import Response

def call_openai(input_list: list[dict]):

    response = client.responses.create(
        model="gpt-4o-mini",
        tools=tools,
        prompt={
            "id": os.getenv("PROMPT_ID"),
            "variables": {
                "resume_pdf": {
                    "type": "input_file",
                    "file_id": resume_pdf_file.id
                },
                "linkedin_pdf": {
                    "type": "input_file",
                    "file_id": linkedin_pdf_file.id
                }
            }
        },
        input=input_list
    )

    return response


def handle_openai_response(response: Response, history) -> str:
    output = response.output[0]

    if output.type == "function_call":
        return call_tool(output, history)

    elif output.type == "message":
        return response.output_text

    else:
        return "unknown_response_type"

def call_tool(output: ResponseCustomToolCall, history) -> str:
    arguments = json.loads(output.arguments)

    if output.name == "record_unknown_question":
        return record_unknown_question(arguments["question"])
    if output.name == "record_user_details":
        return record_user_details(arguments["email"], arguments["name"], arguments["notes"])
    
    return "unknown_tool"



### Launch the chat interface

In [29]:
def get_history(history):
    history_list = []
    for message in history:
        for c in message["content"]:
            history_list.append({"role": message["role"], "content": c["text"]})
    return history_list

def chat(message, history):
    messages = get_history(history) + [{"role": "user", "content": message}]
    response = call_openai(messages)
    output_text = handle_openai_response(response, history)
    
    if output_text == "recorded_successfully":
        messages = messages + response.output + [{
            "type": "function_call_output",
            "call_id": response.output[0].call_id,
            "output": output_text
        }]
        response = call_openai(messages)
        output_text = handle_openai_response(response, history)

    return output_text


gr.ChatInterface(fn=chat, title="Chat with Shriharsha's professional twin").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/Shriharsha.Kowshika/Projects/my_professional_twin/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Shriharsha.Kowshika/Projects/my_professional_twin/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Shriharsha.Kowshika/Projects/my_professional_twin/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2147, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Shriharsha.Kowshika/Projects/my_professional_twin/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1627, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi